<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [38]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])
await piplite.install(['geopy'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [5]:
## Task 1: Mark all launch sites on a map

# Example data
data = {
    'Launch Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-41'],
    'Latitude': [28.5623, 34.6328, 28.6084, 28.5632],
    'Longitude': [-80.5774, -120.6108, -80.6043, -80.5768]
}
launch_sites_df = pd.DataFrame(data)


In [6]:

# Create a base map centered around the average coordinates
center_lat = launch_sites_df['Latitude'].mean()
center_lon = launch_sites_df['Longitude'].mean()
launch_map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Add a marker for each launch site
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(launch_map)

# Display the map (use launch_map.save if you're running this outside of Jupyter)
launch_map


In [7]:
launch_map.save('launch_sites_map.html')


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [8]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [12]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
import folium
from folium.features import DivIcon

# Center map around NASA coordinate
nasa_coordinate = [28.562302, -80.577356]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# List of launch sites with coordinates
launch_sites = [
    ['CCAFS LC-40', 28.562302, -80.577356],
    ['CCAFS SLC-40', 28.563197, -80.576820],
    ['KSC LC-39A', 28.573255, -80.646895],
    ['VAFB SLC-4E', 34.632834, -120.610740]
]

# Add both circle and label for each site
for site, lat, lon in launch_sites:
    folium.Circle(
        location=[lat, lon],
        radius=500,
        color='blue',
        fill=True,
        fill_color='lightblue',
        popup=folium.Popup(site, parse_html=True)
    ).add_to(site_map)

    # Add text label directly on the map
    folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 6pt; color: darkblue;"><b>{site}</b></div>',
        )
    ).add_to(site_map)

site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Assign color based on success or failure
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Preview the updated DataFrame
spacex_df[['Launch Site', 'class', 'marker_color']].head()



,Launch Site,class,marker_color
0,CCAFS LC-40,0,red
1,CCAFS LC-40,0,red
2,CCAFS LC-40,0,red
3,CCAFS LC-40,0,red
4,CCAFS LC-40,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [16]:
# This might fail in Skills Network Lab
from folium.plugins import MarkerCluster

In [22]:
marker_cluster = MarkerCluster()
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failed'}",
        icon=folium.Icon(
            color='white', 
            icon_color=record['marker_color']
        )
    )
marker_cluster.add_child(marker)
site_map   

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [65]:
import folium
print(folium.__version__)

0.20.0


In [23]:
from folium.plugins import MousePosition

# Simplified mouse position without custom formatter
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    prefix='Lat: '
)
site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [27]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [25]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [31]:
# Launch site coordinates
launch_site_lat = 28.56367
launch_site_lon = -80.57163

# Closest coastline point (Atlantic Ocean, directly east)
coastline_lat = 28.56367  # Same latitude as launch site
coastline_lon = -80.52    # Approximate longitude of Atlantic coastline

# Calculate distance
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create the distance marker for coastline
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

# Add the marker to existing site_map
site_map.add_child(distance_marker)

# Optional: Add a regular marker for the coastline point for better visibility
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup=f"Closest Coastline Point<br>Distance: {distance:.2f} km",
    tooltip="Atlantic Ocean Coastline",
    icon=folium.Icon(color='blue', icon='tint', prefix='fa')
)
site_map.add_child(coastline_marker)

# Optional: Add a line connecting launch site to coastline
coastline_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='red',
    weight=2,
    opacity=0.7,
    popup=f"Distance to Coast: {distance:.2f} km"
)
site_map.add_child(coastline_line)

# Alternative: Multiple coastline points for comparison
coastline_points = [
    # Cape Canaveral Beach
    (28.396837, -80.605659, "Cape Canaveral Beach"),
    # Cocoa Beach
    (28.3200, -80.6076, "Cocoa Beach"),
    # Direct East (Atlantic Ocean)
    (28.56367, -80.52, "Atlantic Ocean (Direct East)"),
    # Titusville area coastline
    (28.6135, -80.8007, "Titusville Coast")
]

# Add markers for all coastline points
for coast_lat, coast_lon, name in coastline_points:
    dist = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)
    
    # Distance marker with custom styling
    distance_marker = folium.Marker(
        [coast_lat, coast_lon],
        icon=DivIcon(
            icon_size=(50, 20),
            icon_anchor=(25, 10),
            html=f'<div style="font-size: 10px; color:#d35400; background-color:white; border:1px solid #d35400; padding:2px; border-radius:3px;"><b>{dist:.2f} KM</b></div>',
        )
    )
    site_map.add_child(distance_marker)
    
    # Regular marker for the coastline point
    coast_marker = folium.Marker(
        [coast_lat, coast_lon],
        popup=f"{name}<br>Distance: {dist:.2f} km",
        tooltip=name,
        icon=folium.Icon(
            color='lightblue', 
            icon='anchor' if 'Beach' in name else 'tint',
            prefix='fa'
        )
    )
    site_map.add_child(coast_marker)

# Display the map
site_map

# Print results for verification
print("Coastline Distance Analysis:")
print("=" * 30)
print(f"Launch Site: ({launch_site_lat}, {launch_site_lon})")
print(f"Closest Coastline: ({coastline_lat}, {coastline_lon})")
print(f"Distance to Coast: {distance:.2f} km")
print(f"Distance to Coast: {distance * 0.621371:.2f} miles")

# Individual marker creation (as requested in your format)
print("\nIndividual Marker Code:")
print("=" * 25)
coordinate = [coastline_lat, coastline_lon]
distance_marker_example = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)
print("distance_marker created successfully!")

Coastline Distance Analysis:
Launch Site: (28.56367, -80.57163)
Closest Coastline: (28.56367, -80.52)
Distance to Coast: 5.04 km
Distance to Coast: 3.13 miles

Individual Marker Code:
distance_marker created successfully!


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [35]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

launch_site_coord = [28.56367, -80.57163]  
coastline_coord = [28.56367, -80.52]  

# Combine the coordinates in a list: from launch site to coastline
coordinates = [launch_site_coord, coastline_coord]

# Create folium map centered at launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# Create a PolyLine object using the coordinates
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


### Create a marker with distance to a closest city.
### Draw a line between the marker to the launch site

In [40]:
import folium
from geopy.distance import geodesic

# Launch site coordinates
launch_site_coord = [28.56367, -80.57163]

# Coordinates of a nearby city (example: Orlando)
city_coord = [28.5383, -81.3792]
city_name = "Orlando"

# Calculate distance in km
distance_km = geodesic(launch_site_coord, city_coord).km
distance_str = f"Distance to {city_name}: {distance_km:.2f} km"

# Create the map centered at the launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=7)

# Add markers for the launch site and the city
folium.Marker(
    location=launch_site_coord,
    icon=folium.Icon(color='green'),
    popup="Launch Site"
).add_to(site_map)

folium.Marker(
    location=city_coord,
    icon=folium.Icon(color='blue'),
    popup=distance_str
).add_to(site_map)

# Draw a line between the city and the launch site
folium.PolyLine(locations=[launch_site_coord, city_coord], color='purple', weight=2.5).add_to(site_map)

# Display the map (in a Jupyter environment)
site_map


### Create a marker with distance to the closest railway
### Draw a line between the marker to the launch site

In [41]:

# Nearest railway coordinates (example location)
railway_coord = [28.5721, -80.5852]
railway_name = "Railway Line"

# Calculate distance
distance_km = geodesic(launch_site_coord, railway_coord).km
distance_str = f"Distance to {railway_name}: {distance_km:.2f} km"

# Create folium map centered at launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=15)

# Add launch site marker
folium.Marker(
    location=launch_site_coord,
    icon=folium.Icon(color='green'),
    popup="Launch Site"
).add_to(site_map)

# Add railway marker with distance in popup
folium.Marker(
    location=railway_coord,
    icon=folium.Icon(color='red', icon='train', prefix='fa'),
    popup=distance_str
).add_to(site_map)

# Draw line from launch site to railway
folium.PolyLine(locations=[launch_site_coord, railway_coord], color='black', weight=2).add_to(site_map)

# Display the map
site_map


### Create a marker with distance to a closest highway
### Draw a line between the marker to the launch site

In [42]:

# Nearest highway coordinates (example location)
highway_coord = [28.5623, -80.5900]  # Replace with actual highway coordinates
highway_name = "Highway A1A"

# Calculate the distance
distance_km = geodesic(launch_site_coord, highway_coord).km
distance_str = f"Distance to {highway_name}: {distance_km:.2f} km"

# Create folium map centered at the launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=15)

# Add marker for launch site
folium.Marker(
    location=launch_site_coord,
    icon=folium.Icon(color='green'),
    popup="Launch Site"
).add_to(site_map)

# Add marker for highway with distance
folium.Marker(
    location=highway_coord,
    icon=folium.Icon(color='orange', icon='road', prefix='fa'),
    popup=distance_str
).add_to(site_map)

# Draw line between launch site and highway
folium.PolyLine(locations=[launch_site_coord, highway_coord], color='gray', weight=2).add_to(site_map)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
